In [1]:
import requests
import pandas as pd
import time
import os
from config import token
import sqlite3
import warnings
import calendar

In [61]:
def fetch_save_data(url, headers, params, csv_filename):
    results = []
    
    while True:
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['results'])
            results.append(df)
            
            if len(df) < params['limit']:
                break
            
            params['offset'] += params['limit']
        else:
            print("Error:", response.status_code)
            break
    
    df_name = pd.concat(results, ignore_index=True)
    response_count = df_name.shape[0]
    print(f"Response Count for{csv_filename}: {response_count}")
    
    df_name.to_csv(csv_filename, index=False)
    return df_name

# Define the API endpoint URL, headers, pagination parameters, and base CSV filename
api_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"
api_headers = {"token": token}
limit = 1000
offset= 0
datasetid = 'GSOM'
datatypeids = ['PRCP'] #'TMAX','TAVG','TMIN' 'PRCP'
locationid = 'FIPS:US'  # US FIPS code, california is 06

# Define the start and end dates for the range of months you want to fetch
start_year = 2019
start_month =12
end_year = 2019
end_month = 12  # Adjust this based on how many months you want to fetch

# Specify the number of designated iterations (months)
iterations = end_month - start_month + 1

# Loop through the specified range of months
for _ in range(iterations):
    start_date = f"{start_year}-{start_month:02d}-01"
    end_day = calendar.monthrange(start_year, start_month)[1]
    end_date = f"{start_year}-{start_month:02d}-{end_day:02d}"
    
    api_params = {
        'offset': offset,
        'datasetid': datasetid,
        'startdate': start_date,
        'enddate': end_date,
        'locationid': locationid,
        'limit': limit,
        'datatypeid': ','.join(datatypeids)
    }
    
    output_csv_filename = f"Outputs/US_data/prcp_{start_year}_{start_month:02d}.csv" #change between temp and prcp and others
    
    # Call the function with the defined values
    temp_df = fetch_save_data(api_url, api_headers, api_params, output_csv_filename)
    
    # Increment the start_month and start_year for the next iteration
    start_month += 1
    if start_month > 12:
        start_month = 1
        start_year += 1


Response Count forOutputs/US_data/prcp_2019_12.csv: 14848


In [62]:
# Combine all daily CSV files into a single DataFrame
combined_df = pd.concat([pd.read_csv(os.path.join("Outputs/US_data", file)) for file in os.listdir("Outputs/US_data")])

# Save the combined DataFrame as a CSV file
combined_csv_filename = 'Outputs/combined_data.csv'
combined_df.to_csv(combined_csv_filename, index=False)

combined_df.head()

,date,datatype,station,attributes,value
0,2018-01-01T00:00:00,PRCP,GHCND:AQW00061705,",,,W",390.3
1,2018-01-01T00:00:00,PRCP,GHCND:CA001018611,",,,C",117.6
2,2018-01-01T00:00:00,PRCP,GHCND:CA001135126,",,,C",67.6
3,2018-01-01T00:00:00,PRCP,GHCND:CA005020881,",,,C",3.6
4,2018-01-01T00:00:00,PRCP,GHCND:CA006020559,",,,C",21.6


In [63]:
combined_df.shape

(617457, 5)

In [101]:
def fetch_save_data(url, headers, params, csv_filename):
    results = []
    
    while True:
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['results'])
            results.append(df)
            
            if len(df) < params['limit']:
                break
            
            params['offset'] += params['limit']
        else:
            print("Error:", response.status_code)
            break
    
    df_name = pd.concat(results, ignore_index=True)
    response_count = df_name.shape[0]
    print(f"Response Count: {response_count}")
    
    df_name.to_csv(csv_filename, index=False)
    return df_name


In [ ]:
# Define the API endpoint URL, headers, pagination parameters, and CSV filename
api_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/stations"
api_headers = {"token": token}
api_params = {'offset': 0, 
              'limit': 1000,
              'locationid': 'FIPS:US' #CA code is 06
             }
output_csv_filename = 'Outputs/full_station_list.csv'

# Call the function with the defined values
stations_df = fetch_save_data(api_url, api_headers, api_params, output_csv_filename)

stations_df.tail()

In [ ]:
# Define the API endpoint URL, headers, pagination parameters, and CSV filename
api_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"
api_headers = {"token": token}

# Set up pagination parameters
offset = 0
limit = 1000
datasetid = 'GSOM'
datatypeids = ['TAVG','TMIN','TMAX','PRCP']
startdate = '2018-01-01'
enddate = '2018-01-31'
locationid = 'FIPS:US'  # California FIPS code

api_params = {
    'datasetid': datasetid,
    'startdate': startdate,
    'locationid': locationid,
    'enddate': enddate,
    'offset': offset,
    'limit': limit
}

# Combine datatypeids into a comma-separated string
api_params['datatypeid'] = ','.join(datatypeids)

output_csv_filename = 'f(Outputs/US_data/prcp_{year}_{month}.csv)

# Call the function with the defined values
temp_df = fetch_save_data(api_url, api_headers, api_params, output_csv_filename)

# Display the first few rows of the DataFrame
temp_df.tail(10)

In [64]:
# Read the data from your CSV or Excel file
file_path = 'Outputs/combined_data.csv'
data_df = pd.read_csv(file_path)

# Convert 'value' column to float, handling invalid entries as NaN
data_df['value'] = pd.to_numeric(data_df['value'], errors='coerce')

# Initialize empty lists to store the data
dates = []
stations = []
tmax_values = []
tmin_values = []
tavg_values = []
prcp_values = []

# Iterate through the rows of the original DataFrame
for index, row in data_df.iterrows():
    date = row['date']
    station = row['station']
    datatype = row['datatype']
    value = row['value']
    
    if datatype == 'TMAX':
        tmax_values.append(value)
        tmin_values.append(None)
        tavg_values.append(None)
        prcp_values.append(None)
    elif datatype == 'TMIN':
        tmax_values.append(None)
        tmin_values.append(value)
        tavg_values.append(None)
        prcp_values.append(None)
    elif datatype == 'TAVG':
        tmax_values.append(None)
        tmin_values.append(None)
        tavg_values.append(value)
        prcp_values.append(None)
    elif datatype == 'PRCP':
        tmax_values.append(None)
        tmin_values.append(None)
        tavg_values.append(None)
        prcp_values.append(value)
    
    dates.append(date)
    stations.append(station)

# Create a new DataFrame
new_data = {
    'date': dates,
    'station': stations,
    'TMAX': tmax_values,
    'TMIN': tmin_values,
    'TAVG': tavg_values,
    'PRCP': prcp_values
}

new_df = pd.DataFrame(new_data)

# Convert all columns except 'date' and 'station' to float
float_columns = new_df.columns.difference(['date', 'station'])
new_df[float_columns] = new_df[float_columns].astype(float)

# Group by date and station and keep non-null values
grouped_df = new_df.groupby(['date', 'station']).first().reset_index()

# Write the grouped DataFrame to a CSV file
csv_filename = 'Outputs/grouped_df.csv'
grouped_df.to_csv(csv_filename, index=False)


# Print the first few rows of the grouped DataFrame
grouped_df.head()

,date,station,TMAX,TMIN,TAVG,PRCP
0,2018-01-01T00:00:00,GHCND:AQW00061705,30.58,25.12,27.85,390.3
1,2018-01-01T00:00:00,GHCND:CA001018611,7.98,4.63,6.30,117.6
2,2018-01-01T00:00:00,GHCND:CA001135126,1.10,-5.44,-2.17,67.6
3,2018-01-01T00:00:00,GHCND:CA005020881,-8.93,-17.87,-13.40,3.6
4,2018-01-01T00:00:00,GHCND:CA006020559,-8.50,-19.77,-14.14,21.6


In [44]:
grouped_df.describe()


,TMAX,TMIN,TAVG,PRCP
count,42742.000000,42571.000000,42450.000000,180603.000000
mean,18.393210,5.704463,12.037469,91.247268
std,10.955681,10.450637,10.565162,76.867685
min,-13.780000,-27.680000,-20.530000,0.000000
25%,9.430000,-2.590000,3.340000,30.700000
50%,19.760000,5.810000,12.870000,76.500000
75%,28.010000,14.800000,21.380000,132.100000
max,49.190000,35.390000,42.290000,1528.800000


In [45]:
grouped_df = grouped_df.dropna(subset=['TMAX', 'TMIN', 'TAVG','PRCP'])
grouped_df.head()

,date,station,TMAX,TMIN,TAVG,PRCP
0,2018-01-01T00:00:00,GHCND:AQW00061705,30.58,25.12,27.85,390.3
1,2018-01-01T00:00:00,GHCND:CA001018611,7.98,4.63,6.30,117.6
2,2018-01-01T00:00:00,GHCND:CA001135126,1.10,-5.44,-2.17,67.6
3,2018-01-01T00:00:00,GHCND:CA005020881,-8.93,-17.87,-13.40,3.6
4,2018-01-01T00:00:00,GHCND:CA006020559,-8.50,-19.77,-14.14,21.6


In [ ]:
# Folder containing the CSV files
folder = 'Outputs/US_data'

# List of CSV filenames
filenames = ['ca_station_temps_2018_2019_1.csv','ca_station_temps_2018_2019.csv']

# Initialize an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Loop through the filenames and read each CSV into a DataFrame
for filename in filenames:
    file_path = os.path.join(folder, filename)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    else:
        print(f"File not found: {file_path}")

# Remove duplicates from the combined DataFrame
combined_df = combined_df.drop_duplicates()

# Save the combined and deduplicated DataFrame to a new CSV file
output_filename = os.path.join('Outputs/combined_temps.csv')
combined_df.to_csv(output_filename, index=False)

print(f"Combined CSVs and removed duplicates. Saved as '{output_filename}'.")

In [ ]:
# Read the data from your CSV or Excel file
file_path = 'Outputs/combined_temps.csv'
data_df = pd.read_csv(file_path)

# Convert 'value' column to float, handling invalid entries as NaN
data_df['value'] = pd.to_numeric(data_df['value'], errors='coerce')

# Initialize empty lists to store the data
dates = []
stations = []
tmax_values = []
tmin_values = []
tavg_values = []
prcp_values = []

# Iterate through the rows of the original DataFrame
for index, row in data_df.iterrows():
    date = row['date']
    station = row['station']
    datatype = row['datatype']
    value = row['value']
    
    if datatype == 'TMAX':
        tmax_values.append(value)
        tmin_values.append(None)
        tavg_values.append(None)
        prcp_values.append(None)
    elif datatype == 'TMIN':
        tmax_values.append(None)
        tmin_values.append(value)
        tavg_values.append(None)
        prcp_values.append(None)
    elif datatype == 'TAVG':
        tmax_values.append(None)
        tmin_values.append(None)
        tavg_values.append(value)
        prcp_values.append(None)
    elif datatype == 'PRCP':
        tmax_values.append(None)
        tmin_values.append(None)
        tavg_values.append(None)
        prcp_values.append(value)
    
    dates.append(date)
    stations.append(station)

# Create a new DataFrame
new_data = {
    'date': dates,
    'station': stations,
    'TMAX': tmax_values,
    'TMIN': tmin_values,
    'TAVG': tavg_values,
    'PRCP': prcp_values
}

new_df = pd.DataFrame(new_data)

# Convert all columns except 'date' and 'station' to float
float_columns = new_df.columns.difference(['date', 'station'])
new_df[float_columns] = new_df[float_columns].astype(float)

# Group by date and station and keep non-null values
grouped_df = new_df.groupby(['date', 'station']).first().reset_index()

# Drop rows with NaN values in 'TMAX', 'TMIN', and 'TAVG' columns
subset_columns = ['TMAX', 'TMIN', 'TAVG']
grouped_df = grouped_df.dropna(subset=subset_columns, how='all')

# Write the grouped DataFrame to a CSV file
csv_filename = 'Outputs/grouped_df.csv'
grouped_df.to_csv(csv_filename, index=False)

# Print the first few rows of the grouped DataFrame
grouped_df.head()

In [ ]:
# Load the stations.csv file into stations_df
stations_df = pd.read_csv('Outputs/full_station_list.csv')
grouped_df= pd.read_csv('Outputs/grouped_df.csv')

# Merge the two DataFrames based on 'station' using a left join
grouped_df_detailed = pd.merge(grouped_df, stations_df, 
                               left_on='station', right_on='id', how='left')

# Drop the redundant columns (from stations_df)
grouped_df_detailed.drop(columns=['id'], inplace=True)

grouped_df_detailed=grouped_df_detailed[['station','name','latitude','longitude','elevation','date','maxdate','mindate','TAVG','TMAX','TMIN','PRCP']]

# Write the grouped DataFrame to a CSV file
csv_filename = 'Outputs/grouped_df_detailed.csv'
grouped_df_detailed.to_csv(csv_filename, index=False)

grouped_df_detailed.head()

,station,name,latitude,longitude,elevation,date,maxdate,mindate,TAVG,TMAX,TMIN,PRCP
0,GHCND:AQW00061705,"PAGO PAGO WEATHER SERVICE OFFICE AIRPORT, US",-14.33056,-170.71361,3.7,2018-01-01T00:00:00,2023-07-31,1945-08-01,27.85,30.58,25.12,390.3
1,GHCND:CA001018611,"VICTORIA GONZALES CS, WA US",48.03330,-123.33330,70.0,2018-01-01T00:00:00,2023-07-31,1973-01-01,6.30,7.98,4.63,117.6
2,GHCND:CA001135126,"MIDWAY, WA US",49.00000,-118.76670,580.0,2018-01-01T00:00:00,2023-06-11,1987-06-01,-2.17,1.10,-5.44,67.6
3,GHCND:CA005020881,"EMERSON AUTO, ND US",49.00000,-97.23330,242.0,2018-01-01T00:00:00,2023-07-31,2009-07-01,-13.40,-8.93,-17.87,3.6
4,GHCND:CA006020559,"BARWICK, MN US",48.63330,-93.96670,335.0,2018-01-01T00:00:00,2023-07-31,1978-12-01,-14.14,-8.50,-19.77,21.6


In [ ]:
wildfires=pd.read_csv('Resources/data.csv')

In [ ]:
wildfires.columns

Index(['OBJECTID', 'Shape', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE',
       'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
       'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME',
       'ICS_209_PLUS_INCIDENT_JOIN_ID', 'ICS_209_PLUS_COMPLEX_JOIN_ID',
       'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'FIRE_YEAR',
       'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME',
       'NWCG_CAUSE_CLASSIFICATION', 'NWCG_GENERAL_CAUSE',
       'NWCG_CAUSE_AGE_CATEGORY', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME',
       'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_DESCR',
       'STATE', 'COUNTY', 'FIPS_CODE', 'FIPS_NAME'],
      dtype='object')

In [ ]:
wildfires=wildfires[['LATITUDE', 'LONGITUDE', 'FIRE_SIZE','NWCG_REPORTING_UNIT_NAME', 'FIRE_SIZE_CLASS', 'FIRE_YEAR',
       'FPA_ID', 'FIRE_CODE', 'NWCG_CAUSE_CLASSIFICATION',
       'NWCG_GENERAL_CAUSE', 'FIRE_NAME', 'DISCOVERY_DATE', 'CONT_DATE',
       'DISCOVERY_TIME', 'CONT_TIME', 'STATE', 'COUNTY', 'FIPS_CODE'
       ]]
wildfires=wildfires[
    (wildfires['FIRE_YEAR'].isin([2018]))
]
wildfires.head()

,LATITUDE,LONGITUDE,FIRE_SIZE,NWCG_REPORTING_UNIT_NAME,FIRE_SIZE_CLASS,FIRE_YEAR,FPA_ID,FIRE_CODE,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,FIRE_NAME,DISCOVERY_DATE,CONT_DATE,DISCOVERY_TIME,CONT_TIME,STATE,COUNTY,FIPS_CODE
2045714,46.275833,-114.379167,0.1,Bitterroot National Forest,A,2018,FS-6911076,EKS4,Natural,Natural,BLODGETT,8/22/2018,8/22/2018,1625.0,1740.0,MT,081,30081.0
2045715,46.404167,-113.921944,0.1,Bitterroot National Forest,A,2018,FS-6908885,L1RX,Human,Equipment and vehicle use,CORLEY GULCH,7/26/2018,7/28/2018,1225.0,1653.0,MT,081,30081.0
2045716,46.245833,-114.308889,1.0,Bitterroot National Forest,B,2018,FS-6898061,L49X,Human,Recreation and ceremony,CANYON CREEK,9/21/2018,9/23/2018,1305.0,1241.0,MT,081,30081.0
2045717,45.784722,-114.033056,0.1,Bitterroot National Forest,A,2018,FS-6890683,EKS4,Natural,Natural,MAYNARD CREEK,8/17/2018,NaN,1723.0,NaN,MT,081,30081.0
2045718,45.986944,-113.807222,0.1,Bitterroot National Forest,A,2018,FS-6888073,EKS4,Natural,Natural,BLUE,8/12/2018,8/12/2018,1031.0,1334.0,MT,081,30081.0


In [ ]:
wildfires.shape

(80860, 17)

In [ ]:
#Takes 4-10 minutes to run
warnings.filterwarnings("ignore")

wildfires['DISCOVERY_DATE'] = pd.to_datetime(wildfires['DISCOVERY_DATE'])
grouped_df_detailed['date'] = pd.to_datetime(grouped_df_detailed['date'])

def find_nearest_match(row, df, date_col, lat_col, lon_col):
    date_diff = abs((df[date_col] - row['DISCOVERY_DATE']).dt.total_seconds())
    lat_diff = abs(df[lat_col] - row['LATITUDE'])
    lon_diff = abs(df[lon_col] - row['LONGITUDE'])
    total_diff = date_diff + lat_diff + lon_diff
    nearest_idx = total_diff.idxmin()
    return df.loc[nearest_idx]

# Create an empty list to hold the merged rows
merged_rows = []

# Iterate over each row in the filtered wildfires DataFrame
for idx, row in wildfires.iterrows():
    nearest_row = find_nearest_match(row, grouped_df_detailed, 'date', 'latitude', 'longitude')
    merged_row = pd.concat([row, nearest_row])
    merged_rows.append(merged_row)

# Concatenate the list of merged rows into a DataFrame
merged_results = pd.concat(merged_rows, axis=1).T

merged_results=merged_results[['LATITUDE', 'LONGITUDE','COUNTY', 'FIPS_CODE','FIRE_SIZE', 'FIRE_SIZE_CLASS',
       'NWCG_CAUSE_CLASSIFICATION', 'NWCG_GENERAL_CAUSE', 'FIRE_NAME',
       'DISCOVERY_DATE', 'CONT_DATE', 'DISCOVERY_TIME', 'CONT_TIME', 'STATE',
       'station', 'name', 'latitude', 'longitude', 'elevation', 'date', 'TAVG',
       'TMAX', 'TMIN','PRCP']]

merged_results.head(20)

,LATITUDE,LONGITUDE,COUNTY,FIPS_CODE,FIRE_SIZE,FIRE_SIZE_CLASS,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,FIRE_NAME,DISCOVERY_DATE,...,station,name,latitude,longitude,elevation,date,TAVG,TMAX,TMIN,PRCP
0,46.275833,-114.379167,081,30081.0,0.1,A,Natural,Natural,BLODGETT,2018-08-22,...,GHCND:USC00243885,"HAMILTON, MT US",46.24622,-114.16794,1092.7,2018-09-01,12.89,21.95,3.82,3.5
1,46.404167,-113.921944,081,30081.0,0.1,A,Human,Equipment and vehicle use,CORLEY GULCH,2018-07-26,...,GHCND:US1MTRV0009,"STEVENSVILLE 5.7 SSE, MT US",46.427776,-114.058279,1075.9,2018-08-01,NaN,NaN,NaN,12.2
2,46.245833,-114.308889,081,30081.0,1.0,B,Human,Recreation and ceremony,CANYON CREEK,2018-09-21,...,GHCND:USC00243885,"HAMILTON, MT US",46.24622,-114.16794,1092.7,2018-10-01,6.96,13.28,0.65,33.7
3,45.784722,-114.033056,081,30081.0,0.1,A,Natural,Natural,MAYNARD CREEK,2018-08-17,...,GHCND:USC00242221,"DARBY, MT US",46.0263,-114.1763,1182.6,2018-09-01,11.77,20.79,2.74,9.1
4,45.986944,-113.807222,081,30081.0,0.1,A,Natural,Natural,BLUE,2018-08-12,...,GHCND:USC00247967,"SULA 14 NE, MT US",45.911,-113.7394,1571.2,2018-08-01,15.1,26.09,4.12,19.4
5,46.023056,-113.799722,081,30081.0,0.1,A,Natural,Natural,POLLYWOG,2018-08-11,...,GHCND:USC00247967,"SULA 14 NE, MT US",45.911,-113.7394,1571.2,2018-08-01,15.1,26.09,4.12,19.4
6,45.913611,-114.6675,049,16049.0,0.1,A,Natural,Natural,CEDAR,2018-07-25,...,GHCND:USC00108246,"SELWAY LODGE, ID US",46.0081,-114.8442,786.4,2018-08-01,19.38,31.89,6.88,21.1
7,46.121111,-114.239167,081,30081.0,0.1,A,Natural,Natural,DOUBLE STRIKE,2018-06-08,...,GHCND:USC00242221,"DARBY, MT US",46.0263,-114.1763,1182.6,2018-06-01,14.62,21.15,8.09,102.7
8,45.868333,-113.804167,081,30081.0,0.25,A,Natural,Natural,MEADOW,2018-08-17,...,GHCND:USC00247967,"SULA 14 NE, MT US",45.911,-113.7394,1571.2,2018-09-01,9.69,20.79,-1.42,11.2
9,45.914444,-114.635278,049,16049.0,0.1,A,Natural,Natural,MT GEORGE 2,2018-07-24,...,GHCND:USC00108246,"SELWAY LODGE, ID US",46.0081,-114.8442,786.4,2018-08-01,19.38,31.89,6.88,21.1


In [ ]:
name_counts = merged_results['name'].value_counts()

# Print the sum of all counts
total_count = name_counts.sum()
print(f"Total Count: {total_count}")

# Print the unique names and their counts
for name, count in name_counts.items():
    print(f"Name: {name}, Count: {count}")


Total Count: 80860
Name: RED LAKE INDIAN AGCY, MN US, Count: 380
Name: PHOENIX 2.6 NNW, AZ US, Count: 293
Name: PHOENIX 5.4 NE, AZ US, Count: 191
Name: FRESNO 2.4 NW, CA US, Count: 187
Name: PHOENIX 3.8 NNW, AZ US, Count: 187
Name: NEW ROCKFORD 7.0 ESE, ND US, Count: 187
Name: GLENDALE 3.5 S, AZ US, Count: 181
Name: FRESNO 5 NE, CA US, Count: 173
Name: TUCSON 3.0 WNW, AZ US, Count: 169
Name: CAVE CREEK 9.4 WSW, AZ US, Count: 161
Name: ANTHEM 0.9 NE, AZ US, Count: 161
Name: AVONDALE 3.8 NE, AZ US, Count: 160
Name: YUMA 7.7 SE, AZ US, Count: 158
Name: BUNCH 0.8 N, OK US, Count: 150
Name: GLENDALE 2.5 ENE, AZ US, Count: 139
Name: MADERA, CA US, Count: 138
Name: MODESTO 4.1 NW, CA US, Count: 137
Name: TURLOCK NUMBER 2, CA US, Count: 135
Name: PHOENIX 9.2 NE, AZ US, Count: 133
Name: WILLOW CREEK 4.9 NNE, CA US, Count: 128
Name: SWAN FALLS PH, ID US, Count: 127
Name: BAKERSFIELD 5 NW, CA US, Count: 124
Name: MESA 1.3 W, AZ US, Count: 121
Name: BELCOURT KEYA RADIO, ND US, Count: 119
Name: MER

In [ ]:
us_data_2018_2019.columns

Index(['LATITUDE', 'LONGITUDE', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
       'NWCG_CAUSE_CLASSIFICATION', 'DISCOVERY_DATE', 'CONT_DATE', 'STATE',
       'station', 'name', 'latitude', 'longitude', 'elevation', 'date', 'TAVG',
       'TMAX', 'TMIN'],
      dtype='object')

In [ ]:
warnings.filterwarnings("ignore")

us_data_2018_test = merged_results[['FIRE_NAME', 'STATE', 'FIPS_CODE', 'LATITUDE', 'LONGITUDE', 'DISCOVERY_DATE', 'CONT_DATE', 'name', 'latitude', 'longitude', 'date',
                                    'NWCG_CAUSE_CLASSIFICATION', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
                                    'elevation', 'TAVG', 'TMAX', 'TMIN','PRCP']]

# Rename the columns
us_data_2018_test.columns = ['FIRE_NAME', 'STATE', 'FIPS_CODE', 'FIRE_LATITUDE', 'FIRE_LONGITUDE', 'FIRE_DATE', 'CONTAIN_DATE', 'CLOSEST_STATION', 'STATION_LAT', 'STATION_LON', 'READINGS_DATE',
                               'CAUSE_CLASSIFICATION', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
                             'ELEVATION', 'TAVG', 'TMAX', 'TMIN','PRCP']


# Calculate the difference between 'CONTAIN_DATE' and 'FIRE_DATE'
us_data_2018_test['CONTAIN_DATE'] = pd.to_datetime(us_data_2018_test['CONTAIN_DATE'])
us_data_2018_test['DAYS_TO_CONTAIN'] = (us_data_2018_test['CONTAIN_DATE'] - us_data_2018_test['FIRE_DATE']).dt.days

# Replace any NaN values in 'DAYS_TO_CONTAIN' with 1
us_data_2018_test['DAYS_TO_CONTAIN'].fillna(0, inplace=True)

float_columns = {
    'FIRE_LATITUDE': float,
    'FIRE_LONGITUDE': float,
    'STATION_LAT': float,
    'STATION_LON': float,
    'FIRE_SIZE': float,
    'ELEVATION': float,
    'TAVG': float,
    'TMAX': float,
    'TMIN': float,
    'PRCP': float
}

int_columns = {
    'DAYS_TO_CONTAIN': int,
}

csv_filename = 'Outputs/us_data_2018.csv'
us_data_2018_test.to_csv(csv_filename, index=False)

us_data_2018_test = us_data_2018_test.astype({**float_columns, **int_columns})

us_data_2018_test.head(20)

,FIRE_NAME,STATE,FIPS_CODE,FIRE_LATITUDE,FIRE_LONGITUDE,FIRE_DATE,CONTAIN_DATE,CLOSEST_STATION,STATION_LAT,STATION_LON,READINGS_DATE,CAUSE_CLASSIFICATION,FIRE_SIZE,FIRE_SIZE_CLASS,ELEVATION,TAVG,TMAX,TMIN,PRCP,DAYS_TO_CONTAIN
0,BLODGETT,MT,30081.0,46.275833,-114.379167,2018-08-22,2018-08-22,"HAMILTON, MT US",46.246220,-114.167940,2018-09-01,Natural,0.10,A,1092.7,12.89,21.95,3.82,3.5,0
1,CORLEY GULCH,MT,30081.0,46.404167,-113.921944,2018-07-26,2018-07-28,"STEVENSVILLE 5.7 SSE, MT US",46.427776,-114.058279,2018-08-01,Human,0.10,A,1075.9,NaN,NaN,NaN,12.2,2
2,CANYON CREEK,MT,30081.0,46.245833,-114.308889,2018-09-21,2018-09-23,"HAMILTON, MT US",46.246220,-114.167940,2018-10-01,Human,1.00,B,1092.7,6.96,13.28,0.65,33.7,2
3,MAYNARD CREEK,MT,30081.0,45.784722,-114.033056,2018-08-17,NaT,"DARBY, MT US",46.026300,-114.176300,2018-09-01,Natural,0.10,A,1182.6,11.77,20.79,2.74,9.1,0
4,BLUE,MT,30081.0,45.986944,-113.807222,2018-08-12,2018-08-12,"SULA 14 NE, MT US",45.911000,-113.739400,2018-08-01,Natural,0.10,A,1571.2,15.10,26.09,4.12,19.4,0
5,POLLYWOG,MT,30081.0,46.023056,-113.799722,2018-08-11,2018-08-12,"SULA 14 NE, MT US",45.911000,-113.739400,2018-08-01,Natural,0.10,A,1571.2,15.10,26.09,4.12,19.4,1
6,CEDAR,ID,16049.0,45.913611,-114.667500,2018-07-25,2018-07-27,"SELWAY LODGE, ID US",46.008100,-114.844200,2018-08-01,Natural,0.10,A,786.4,19.38,31.89,6.88,21.1,2
7,DOUBLE STRIKE,MT,30081.0,46.121111,-114.239167,2018-06-08,2018-06-08,"DARBY, MT US",46.026300,-114.176300,2018-06-01,Natural,0.10,A,1182.6,14.62,21.15,8.09,102.7,0
8,MEADOW,MT,30081.0,45.868333,-113.804167,2018-08-17,NaT,"SULA 14 NE, MT US",45.911000,-113.739400,2018-09-01,Natural,0.25,A,1571.2,9.69,20.79,-1.42,11.2,0
9,MT GEORGE 2,ID,16049.0,45.914444,-114.635278,2018-07-24,2018-07-26,"SELWAY LODGE, ID US",46.008100,-114.844200,2018-08-01,Natural,0.10,A,786.4,19.38,31.89,6.88,21.1,2


In [ ]:
warnings.filterwarnings("ignore")

us_data_2018 = merged_results[['FIRE_NAME', 'STATE', 'FIPS_CODE', 'LATITUDE', 'LONGITUDE', 'DISCOVERY_DATE', 'CONT_DATE', 'name', 'latitude', 'longitude', 'date',
                                    'NWCG_CAUSE_CLASSIFICATION', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
                                    'elevation', 'TAVG', 'TMAX', 'TMIN']]

# Rename the columns
us_data_2018.columns = ['FIRE_NAME', 'STATE', 'FIPS_CODE', 'FIRE_LATITUDE', 'FIRE_LONGITUDE', 'FIRE_DATE', 'CONTAIN_DATE', 'CLOSEST_STATION', 'STATION_LAT', 'STATION_LON', 'READINGS_DATE',
                               'CAUSE_CLASSIFICATION', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
                             'ELEVATION', 'TAVG', 'TMAX', 'TMIN']


# Calculate the difference between 'CONTAIN_DATE' and 'FIRE_DATE'
us_data_2018['CONTAIN_DATE'] = pd.to_datetime(us_data_2018['CONTAIN_DATE'])
us_data_2018['DAYS_TO_CONTAIN'] = (us_data_2018['CONTAIN_DATE'] - us_data_2018['FIRE_DATE']).dt.days

# Replace any NaN values in 'DAYS_TO_CONTAIN' with 1
us_data_2018['DAYS_TO_CONTAIN'].fillna(0, inplace=True)

float_columns = {
    'FIRE_LATITUDE': float,
    'FIRE_LONGITUDE': float,
    'STATION_LAT': float,
    'STATION_LON': float,
    'FIRE_SIZE': float,
    'ELEVATION': float,
    'TAVG': float,
    'TMAX': float,
    'TMIN': float
}

int_columns = {
    'DAYS_TO_CONTAIN': int,
}

us_data_2018 = us_data_2018.astype({**float_columns, **int_columns})

csv_filename = 'Outputs/us_data_2018.csv'
us_data_2018.to_csv(csv_filename, index=False)

us_data_2018.head(20)

,FIRE_NAME,STATE,FIPS_CODE,FIRE_LATITUDE,FIRE_LONGITUDE,FIRE_DATE,CONTAIN_DATE,CLOSEST_STATION,STATION_LAT,STATION_LON,READINGS_DATE,CAUSE_CLASSIFICATION,FIRE_SIZE,FIRE_SIZE_CLASS,ELEVATION,TAVG,TMAX,TMIN,DAYS_TO_CONTAIN
0,BLODGETT,MT,30081.0,46.275833,-114.379167,2018-08-22,2018-08-22,"HAMILTON, MT US",46.24622,-114.16794,2018-09-01,Natural,0.10,A,1092.7,12.89,21.95,3.82,0
1,CORLEY GULCH,MT,30081.0,46.404167,-113.921944,2018-07-26,2018-07-28,"STEVENSVILLE, MT US",46.51370,-114.09100,2018-08-01,Human,0.10,A,1028.7,18.29,28.05,8.53,2
2,CANYON CREEK,MT,30081.0,46.245833,-114.308889,2018-09-21,2018-09-23,"HAMILTON, MT US",46.24622,-114.16794,2018-10-01,Human,1.00,B,1092.7,6.96,13.28,0.65,2
3,MAYNARD CREEK,MT,30081.0,45.784722,-114.033056,2018-08-17,NaT,"DARBY, MT US",46.02630,-114.17630,2018-09-01,Natural,0.10,A,1182.6,11.77,20.79,2.74,0
4,BLUE,MT,30081.0,45.986944,-113.807222,2018-08-12,2018-08-12,"SULA 14 NE, MT US",45.91100,-113.73940,2018-08-01,Natural,0.10,A,1571.2,15.10,26.09,4.12,0
5,POLLYWOG,MT,30081.0,46.023056,-113.799722,2018-08-11,2018-08-12,"SULA 14 NE, MT US",45.91100,-113.73940,2018-08-01,Natural,0.10,A,1571.2,15.10,26.09,4.12,1
6,CEDAR,ID,16049.0,45.913611,-114.667500,2018-07-25,2018-07-27,"SELWAY LODGE, ID US",46.00810,-114.84420,2018-08-01,Natural,0.10,A,786.4,19.38,31.89,6.88,2
7,DOUBLE STRIKE,MT,30081.0,46.121111,-114.239167,2018-06-08,2018-06-08,"DARBY, MT US",46.02630,-114.17630,2018-06-01,Natural,0.10,A,1182.6,14.62,21.15,8.09,0
8,MEADOW,MT,30081.0,45.868333,-113.804167,2018-08-17,NaT,"SULA 14 NE, MT US",45.91100,-113.73940,2018-09-01,Natural,0.25,A,1571.2,9.69,20.79,-1.42,0
9,MT GEORGE 2,ID,16049.0,45.914444,-114.635278,2018-07-24,2018-07-26,"SELWAY LODGE, ID US",46.00810,-114.84420,2018-08-01,Natural,0.10,A,786.4,19.38,31.89,6.88,2


In [ ]:
csv_filename = 'Outputs/us_data_2018.csv'
us_data_2018.to_csv(csv_filename, index=False)

NameError: name 'us_data_2018' is not defined